## import pyspark

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=99105391b444765d5c1f5c727c962a04e3f33234b529392e6118c835be6313c8
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


## SparkSession intialization

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SQL_window_functions").getOrCreate()

# Introduction to Window Functions:

-  Window functions are analytics functions that  are used to perform aggregate
 operations over a specified set of rows called window.
-  These are similar to groupby, but these functions give result for all the rows in window where as groupby give result for grouped data.
-  window function calculate value for the set of rows that are defined by window specification.
-  window specification is defined by over() clause.
-  over clause requires partition and order by
- partition divides the data into window and order by sorts the window.

In [4]:
#sample data
data = [("James","CSE",60), ("Michael","CSE",70),
        ("Robert","CSE",80), ("Maria","CSE",75),
        ("Jen","ECE",65),("Scott","ECE",85),
        ("Smith","ECE",90),("John","ECE",95),
        ("Williams","MECH",78),("Rosy","MECH",68),
        ("Ramesh","MECH",88),("Joseph","MECH",98),
        ("Julia","CIVIL",58),("Thomas","CIVIL",68),
        ("Charles","CIVIL",78),("Lisa","CIVIL",88),
        ("Kevin","EEE",93),("Daniel","EEE",83),
        ("Brian","EEE",73),("George","EEE",63)]

columns= ["name","branch","marks"]
df = spark.createDataFrame(data = data, schema = columns)
df.show()

df = df.createOrReplaceTempView('students')


+--------+------+-----+
|    name|branch|marks|
+--------+------+-----+
|   James|   CSE|   60|
| Michael|   CSE|   70|
|  Robert|   CSE|   80|
|   Maria|   CSE|   75|
|     Jen|   ECE|   65|
|   Scott|   ECE|   85|
|   Smith|   ECE|   90|
|    John|   ECE|   95|
|Williams|  MECH|   78|
|    Rosy|  MECH|   68|
|  Ramesh|  MECH|   88|
|  Joseph|  MECH|   98|
|   Julia| CIVIL|   58|
|  Thomas| CIVIL|   68|
| Charles| CIVIL|   78|
|    Lisa| CIVIL|   88|
|   Kevin|   EEE|   93|
|  Daniel|   EEE|   83|
|   Brian|   EEE|   73|
|  George|   EEE|   63|
+--------+------+-----+



### groupby

In [11]:
spark.sql("""

        select branch,avg(marks)
        from students
        group by (branch);

""").show()

+------+----------+
|branch|avg(marks)|
+------+----------+
|   ECE|     83.75|
|   CSE|     71.25|
|  MECH|      83.0|
| CIVIL|      73.0|
|   EEE|      78.0|
+------+----------+



### window function

In [14]:
spark.sql("""
  select *,
  avg(marks) over() as avg_marks
  from students

""").show()

+--------+------+-----+---------+
|    name|branch|marks|avg_marks|
+--------+------+-----+---------+
|   James|   CSE|   60|     77.8|
| Michael|   CSE|   70|     77.8|
|  Robert|   CSE|   80|     77.8|
|   Maria|   CSE|   75|     77.8|
|     Jen|   ECE|   65|     77.8|
|   Scott|   ECE|   85|     77.8|
|   Smith|   ECE|   90|     77.8|
|    John|   ECE|   95|     77.8|
|Williams|  MECH|   78|     77.8|
|    Rosy|  MECH|   68|     77.8|
|  Ramesh|  MECH|   88|     77.8|
|  Joseph|  MECH|   98|     77.8|
|   Julia| CIVIL|   58|     77.8|
|  Thomas| CIVIL|   68|     77.8|
| Charles| CIVIL|   78|     77.8|
|    Lisa| CIVIL|   88|     77.8|
|   Kevin|   EEE|   93|     77.8|
|  Daniel|   EEE|   83|     77.8|
|   Brian|   EEE|   73|     77.8|
|  George|   EEE|   63|     77.8|
+--------+------+-----+---------+



In [13]:
spark.sql("""
  select *,
  avg(marks) over(partition by branch) as avg_marks
  from students

""").show()

+--------+------+-----+---------+
|    name|branch|marks|avg_marks|
+--------+------+-----+---------+
|   Julia| CIVIL|   58|     73.0|
|  Thomas| CIVIL|   68|     73.0|
| Charles| CIVIL|   78|     73.0|
|    Lisa| CIVIL|   88|     73.0|
|   James|   CSE|   60|    71.25|
| Michael|   CSE|   70|    71.25|
|  Robert|   CSE|   80|    71.25|
|   Maria|   CSE|   75|    71.25|
|     Jen|   ECE|   65|    83.75|
|   Scott|   ECE|   85|    83.75|
|   Smith|   ECE|   90|    83.75|
|    John|   ECE|   95|    83.75|
|   Kevin|   EEE|   93|     78.0|
|  Daniel|   EEE|   83|     78.0|
|   Brian|   EEE|   73|     78.0|
|  George|   EEE|   63|     78.0|
|Williams|  MECH|   78|     83.0|
|    Rosy|  MECH|   68|     83.0|
|  Ramesh|  MECH|   88|     83.0|
|  Joseph|  MECH|   98|     83.0|
+--------+------+-----+---------+



In [15]:
spark.sql("""
  select *,
  avg(marks) over(partition by branch order by name) as avg_marks
  from students

""").show()

+--------+------+-----+-----------------+
|    name|branch|marks|        avg_marks|
+--------+------+-----+-----------------+
| Charles| CIVIL|   78|             78.0|
|   Julia| CIVIL|   58|             68.0|
|    Lisa| CIVIL|   88|74.66666666666667|
|  Thomas| CIVIL|   68|             73.0|
|   James|   CSE|   60|             60.0|
|   Maria|   CSE|   75|             67.5|
| Michael|   CSE|   70|68.33333333333333|
|  Robert|   CSE|   80|            71.25|
|     Jen|   ECE|   65|             65.0|
|    John|   ECE|   95|             80.0|
|   Scott|   ECE|   85|81.66666666666667|
|   Smith|   ECE|   90|            83.75|
|   Brian|   EEE|   73|             73.0|
|  Daniel|   EEE|   83|             78.0|
|  George|   EEE|   63|             73.0|
|   Kevin|   EEE|   93|             78.0|
|  Joseph|  MECH|   98|             98.0|
|  Ramesh|  MECH|   88|             93.0|
|    Rosy|  MECH|   68|84.66666666666667|
|Williams|  MECH|   78|             83.0|
+--------+------+-----+-----------

### Use cases of Window functions:


-  **Ranking and Row Numbering:**

 Assign ranks to rows based on specific criteria.
Generate unique row numbers for pagination.

-  **Data Partitioning:**

  Calculate averages, sums, and other aggregates within logical groups.
  
-  **Running Totals and Cumulative Statistics:**

 Compute running totals and cumulative sums.

 Calculate moving averages and running counts.

-  **Comparing Rows Within Groups:**

 Compare current row values with previous or following rows.

-  **First and Last Value Analysis:**

  Identify the first or last value within a partition.

-  **Gap and Island Analysis:**

  Detect gaps or continuous ranges in sequences.

-  **Percentile and Distribution Analysis:**

  Calculate percentiles and distribution metrics.

-  **Top-N Queries:**

  Retrieve the top N items within each group.

-  **Time Series Analysis:**

  Analyze time series data with moving averages or period comparisons.

- **Pivoting Rows to Columns:**

  Transform rows into columns for reporting.

-  **Advanced Analytical Queries:**

  Perform complex queries involving multiple rows of a table.

-  **Data Smoothing:**

  Smooth out data fluctuations to reveal trends.


In [22]:
#practice problems

# 1.extract the students who marks greater than their avg branch marks

spark.sql("""
          select *
          from (
                   select *,
                    avg(marks) over(partition by branch order by name) as branch_avg_marks
                    from students) as t
          where t.marks > branch_avg_marks

""").show()

+-------+------+-----+-----------------+
|   name|branch|marks| branch_avg_marks|
+-------+------+-----+-----------------+
|   Lisa| CIVIL|   88|74.66666666666667|
|  Maria|   CSE|   75|             67.5|
|Michael|   CSE|   70|68.33333333333333|
| Robert|   CSE|   80|            71.25|
|   John|   ECE|   95|             80.0|
|  Scott|   ECE|   85|81.66666666666667|
|  Smith|   ECE|   90|            83.75|
| Daniel|   EEE|   83|             78.0|
|  Kevin|   EEE|   93|             78.0|
+-------+------+-----+-----------------+



## Ranking Functions:

*  RANK() :

  -  Assigns a unique rank to each row within a partition, with gaps in the ranking for tied values.
  -  Rows with the same values in the order by clause will receive the same rank, and the next rank will be incremented by the number of tied rows.
  -  Example: If two rows tie for rank 1, the next row will receive rank 3.

*  DENSE_RANK():

 -  Similar to RANK(), but without gaps in the ranking sequence for tied values.
 -  Rows with the same values will receive the same rank, and the next distinct value will receive the next consecutive rank.
 - Example: If two rows tie for rank 1, the next row will receive rank 2.


*  ROW_NUMBER():

  - Assigns a unique number to each row within a partition, starting at 1 and incrementing by 1 for each row, regardless of ties.
  - Unlike RANK() and DENSE_RANK(), ROW_NUMBER() does not assign the same number to tied rows; each row gets a distinct number based on the order.
  - Example: Even if two rows have the same values, they will receive different row numbers (e.g., 1 and 2).

*  NTILE():
  - Distributes the rows in an ordered partition into a specified number of approximately equal groups, or tiles.
  - The argument n specifies the number of groups to divide the rows into.
  - Rows are distributed as evenly as possible into the groups, and the groups are numbered starting at 1.
   - Example: If there are 10 rows and you use NTILE(4), the function will attempt to distribute the rows into 4 groups with 2 or 3 rows each

In [34]:
spark.sql("""

    select *,
    rank() over(order by marks desc) as rank,
    dense_rank() over(order by marks desc) as dense_rank,
    row_number() over(partition by branch order by branch) as brach_row_number

    from students
""").show()

+--------+------+-----+----+----------+----------------+
|    name|branch|marks|rank|dense_rank|brach_row_number|
+--------+------+-----+----+----------+----------------+
|    Lisa| CIVIL|   88|   5|         5|               1|
| Charles| CIVIL|   78|  10|         9|               2|
|  Thomas| CIVIL|   68|  15|        13|               3|
|   Julia| CIVIL|   58|  20|        17|               4|
|  Robert|   CSE|   80|   9|         8|               1|
|   Maria|   CSE|   75|  12|        10|               2|
| Michael|   CSE|   70|  14|        12|               3|
|   James|   CSE|   60|  19|        16|               4|
|    John|   ECE|   95|   2|         2|               1|
|   Smith|   ECE|   90|   4|         4|               2|
|   Scott|   ECE|   85|   7|         6|               3|
|     Jen|   ECE|   65|  17|        14|               4|
|   Kevin|   EEE|   93|   3|         3|               1|
|  Daniel|   EEE|   83|   8|         7|               2|
|   Brian|   EEE|   73|  13|   

In [37]:
spark.sql("""

    select *,
    ntile(5) over(order by branch) as groups
    from students
""").show()

+--------+------+-----+------+
|    name|branch|marks|groups|
+--------+------+-----+------+
|   Julia| CIVIL|   58|     1|
|  Thomas| CIVIL|   68|     1|
| Charles| CIVIL|   78|     1|
|    Lisa| CIVIL|   88|     1|
|   James|   CSE|   60|     2|
| Michael|   CSE|   70|     2|
|  Robert|   CSE|   80|     2|
|   Maria|   CSE|   75|     2|
|     Jen|   ECE|   65|     3|
|   Scott|   ECE|   85|     3|
|   Smith|   ECE|   90|     3|
|    John|   ECE|   95|     3|
|   Kevin|   EEE|   93|     4|
|  Daniel|   EEE|   83|     4|
|   Brian|   EEE|   73|     4|
|  George|   EEE|   63|     4|
|Williams|  MECH|   78|     5|
|    Rosy|  MECH|   68|     5|
|  Ramesh|  MECH|   88|     5|
|  Joseph|  MECH|   98|     5|
+--------+------+-----+------+



In [39]:
#practice problem

orders = spark.read.csv('/content/orders.csv',inferSchema=True,header=True)
orders.createOrReplaceTempView('orders')

In [41]:
spark.sql("""
    describe orders
""").show()



+-----------------+---------+-------+
|         col_name|data_type|comment|
+-----------------+---------+-------+
|         order_id|      int|   NULL|
|          user_id|      int|   NULL|
|             r_id|      int|   NULL|
|           amount|      int|   NULL|
|             date|     date|   NULL|
|       partner_id|      int|   NULL|
|    delivery_time|      int|   NULL|
|  delivery_rating|      int|   NULL|
|restaurant_rating|      int|   NULL|
+-----------------+---------+-------+



In [86]:
spark.sql("""

  with
      cte as(

          select month(date),user_id,sum(amount) from orders
          group by month(date),user_id
          order by month(date)
        ),

      transformed_df as(

          select `month(date)` as  month_num,
                  `sum(amount)` as total_spend,
                  user_id
          from cte
        ),

    result as(
    select *,
    rank() over(partition by month_num order by total_spend desc) as rank
    from transformed_df)

    select * from result
    where rank<3

""").show()

+---------+-----------+-------+----+
|month_num|total_spend|user_id|rank|
+---------+-----------+-------+----+
|        5|        965|      1|   1|
|        5|        860|      3|   2|
|        6|       1480|      2|   1|
|        6|        800|      4|   2|
|        7|       3035|      5|   1|
|        7|       1190|      2|   2|
+---------+-----------+-------+----+



## Value functions

- FIRST_VALUE():

  This function returns the first value in an ordered set of values.

- LAST_VALUE():

  - This function returns the last value in an ordered set of values.

  **Problem when using without frame:**
  - By default, the window frame for LAST_VALUE() is the range between the start of the partition and the current row.
  - Without specifying a frame, the "last value" might not be the actual last value of the partition but rather the last value up to the current row.

  Frame in a window function is a subset of rows within the partition that determines the scope of the window function calculation.
  
  The frame is defined using a combination of two clauses in the window function: ROWS and BETWEEN.
  
  The ROWS clause specifies how many rows should be included in the frame relative to the current row.For example,**ROWS 3 PRECEDING** means that the frame includes the current row
  and the three rows that precede it in the partition.
  
  The BETWEEN clause specifies the boundaries of the frame.

 Examples

 **ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW**
  - means that the frame includes all rows from the beginning of the partition up to and including the
current row.

 **ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING:**
 the frame includes the current row and the row immediately before and after it.

 **ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING:** the frame includes all rows in the partition.

 **ROWS BETWEEN 3 PRECEDING AND 2 FOLLOWING:**
  the frame includes the current row and the three rows before it and the two rows after it.


- NTH_VALUE():

  - The NTH_VALUE() function is a window function in SQL that returns the value of the n-th row within a window (partition) of a result set.
  -This function is useful when you want to retrieve a specific value from a set of rows, particularly in cases where you need to reference a value that is not necessarily the first or last within a group.




In [91]:
#first_value

spark.sql("""

    select *,
    first_value(name) over(order by marks desc) as overall_topper,
    first_value(name) over(partition by branch order by marks desc) as brach_topper
    from students

""").show()

+--------+------+-----+--------------+------------+
|    name|branch|marks|overall_topper|brach_topper|
+--------+------+-----+--------------+------------+
|    Lisa| CIVIL|   88|        Joseph|        Lisa|
| Charles| CIVIL|   78|        Joseph|        Lisa|
|  Thomas| CIVIL|   68|        Joseph|        Lisa|
|   Julia| CIVIL|   58|        Joseph|        Lisa|
|  Robert|   CSE|   80|        Joseph|      Robert|
|   Maria|   CSE|   75|        Joseph|      Robert|
| Michael|   CSE|   70|        Joseph|      Robert|
|   James|   CSE|   60|        Joseph|      Robert|
|    John|   ECE|   95|        Joseph|        John|
|   Smith|   ECE|   90|        Joseph|        John|
|   Scott|   ECE|   85|        Joseph|        John|
|     Jen|   ECE|   65|        Joseph|        John|
|   Kevin|   EEE|   93|        Joseph|       Kevin|
|  Daniel|   EEE|   83|        Joseph|       Kevin|
|   Brian|   EEE|   73|        Joseph|       Kevin|
|  George|   EEE|   63|        Joseph|       Kevin|
|  Joseph|  

In [105]:
#last_value()


spark.sql("""

    select *,
    last_value(name) over(order by marks) as last_ranker
    from students

""").show()

#problem: the window frame for LAST_VALUE() is the range between the start of the partition and the current row.
#           so it is not giving legitimate values.

+--------+------+-----+-----------+
|    name|branch|marks|last_ranker|
+--------+------+-----+-----------+
|   Julia| CIVIL|   58|      Julia|
|   James|   CSE|   60|      James|
|  George|   EEE|   63|     George|
|     Jen|   ECE|   65|        Jen|
|    Rosy|  MECH|   68|     Thomas|
|  Thomas| CIVIL|   68|     Thomas|
| Michael|   CSE|   70|    Michael|
|   Brian|   EEE|   73|      Brian|
|   Maria|   CSE|   75|      Maria|
|Williams|  MECH|   78|    Charles|
| Charles| CIVIL|   78|    Charles|
|  Robert|   CSE|   80|     Robert|
|  Daniel|   EEE|   83|     Daniel|
|   Scott|   ECE|   85|      Scott|
|  Ramesh|  MECH|   88|       Lisa|
|    Lisa| CIVIL|   88|       Lisa|
|   Smith|   ECE|   90|      Smith|
|   Kevin|   EEE|   93|      Kevin|
|    John|   ECE|   95|       John|
|  Joseph|  MECH|   98|     Joseph|
+--------+------+-----+-----------+



In [104]:
#last_value() with frame

spark.sql("""

    select *,
    last_value(name)
    over(order by marks desc
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as overall_last_ranker,

    last_value(marks)
    over(order by marks desc
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as overall_lowest_marks,

    last_value(name)
    over(partition by branch order by marks desc
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as branch_last_ranker,

    last_value(marks)
    over(partition by branch order by marks desc
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as branch_lowest_marks
    from students

""").show()


+--------+------+-----+-------------------+--------------------+------------------+-------------------+
|    name|branch|marks|overall_last_ranker|overall_lowest_marks|branch_last_ranker|branch_lowest_marks|
+--------+------+-----+-------------------+--------------------+------------------+-------------------+
|    Lisa| CIVIL|   88|              Julia|                  58|             Julia|                 58|
| Charles| CIVIL|   78|              Julia|                  58|             Julia|                 58|
|  Thomas| CIVIL|   68|              Julia|                  58|             Julia|                 58|
|   Julia| CIVIL|   58|              Julia|                  58|             Julia|                 58|
|  Robert|   CSE|   80|              Julia|                  58|             James|                 60|
|   Maria|   CSE|   75|              Julia|                  58|             James|                 60|
| Michael|   CSE|   70|              Julia|                  58|

In [110]:
#nth_value()
# NTH_VALUE(column_name, n) OVER (
#     [PARTITION BY partition_expression]
#     ORDER BY order_expression
#     [ROWS or RANGE clause]
# ) AS alias_name


spark.sql("""

    select * ,
      nth_value(name,2) over(order by marks desc) as overall_second_ranker

    from students

""").show()

#same problem due to frame

+--------+------+-----+---------------------+
|    name|branch|marks|overall_second_ranker|
+--------+------+-----+---------------------+
|  Joseph|  MECH|   98|                 NULL|
|    John|   ECE|   95|                 John|
|   Kevin|   EEE|   93|                 John|
|   Smith|   ECE|   90|                 John|
|  Ramesh|  MECH|   88|                 John|
|    Lisa| CIVIL|   88|                 John|
|   Scott|   ECE|   85|                 John|
|  Daniel|   EEE|   83|                 John|
|  Robert|   CSE|   80|                 John|
|Williams|  MECH|   78|                 John|
| Charles| CIVIL|   78|                 John|
|   Maria|   CSE|   75|                 John|
|   Brian|   EEE|   73|                 John|
| Michael|   CSE|   70|                 John|
|    Rosy|  MECH|   68|                 John|
|  Thomas| CIVIL|   68|                 John|
|     Jen|   ECE|   65|                 John|
|  George|   EEE|   63|                 John|
|   James|   CSE|   60|           

In [112]:
spark.sql("""

    select * ,
      nth_value(name,2) over(order by marks desc
      ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as overall_second_ranker,

      nth_value(name,2) over(partition by branch order by marks desc
      ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as branch_second_ranker

    from students

""").show()

+--------+------+-----+---------------------+--------------------+
|    name|branch|marks|overall_second_ranker|branch_second_ranker|
+--------+------+-----+---------------------+--------------------+
|    Lisa| CIVIL|   88|                 John|             Charles|
| Charles| CIVIL|   78|                 John|             Charles|
|  Thomas| CIVIL|   68|                 John|             Charles|
|   Julia| CIVIL|   58|                 John|             Charles|
|  Robert|   CSE|   80|                 John|               Maria|
|   Maria|   CSE|   75|                 John|               Maria|
| Michael|   CSE|   70|                 John|               Maria|
|   James|   CSE|   60|                 John|               Maria|
|    John|   ECE|   95|                 John|               Smith|
|   Smith|   ECE|   90|                 John|               Smith|
|   Scott|   ECE|   85|                 John|               Smith|
|     Jen|   ECE|   65|                 John|               Sm

## LEAD() and LAG()

  window functions that allow you to access data from a subsequent or preceding row in the same result set, without needing to perform a self-join. These functions are particularly useful for comparing values between consecutive rows.

In [135]:
#lead()

spark.sql("""

    select *,
    lead(marks) over(order by marks) as lead,
    lag(marks) over(order by marks) as lag
    from students

""").show()

+--------+------+-----+----+----+
|    name|branch|marks|lead| lag|
+--------+------+-----+----+----+
|   Julia| CIVIL|   58|  60|NULL|
|   James|   CSE|   60|  63|  58|
|  George|   EEE|   63|  65|  60|
|     Jen|   ECE|   65|  68|  63|
|    Rosy|  MECH|   68|  68|  65|
|  Thomas| CIVIL|   68|  70|  68|
| Michael|   CSE|   70|  73|  68|
|   Brian|   EEE|   73|  75|  70|
|   Maria|   CSE|   75|  78|  73|
|Williams|  MECH|   78|  78|  75|
| Charles| CIVIL|   78|  80|  78|
|  Robert|   CSE|   80|  83|  78|
|  Daniel|   EEE|   83|  85|  80|
|   Scott|   ECE|   85|  88|  83|
|  Ramesh|  MECH|   88|  88|  85|
|    Lisa| CIVIL|   88|  90|  88|
|   Smith|   ECE|   90|  93|  88|
|   Kevin|   EEE|   93|  95|  90|
|    John|   ECE|   95|  98|  93|
|  Joseph|  MECH|   98|NULL|  95|
+--------+------+-----+----+----+



In [128]:
#lag()

spark.sql("""

  select *,
  lag(t.total_revenue) over(order by t.month_num) as lag,
  (((t.total_revenue - lag(t.total_revenue) over(order by t.month_num))/lag(t.total_revenue) over(order by t.month_num))*100) as growth_per
  from
      (select
            month(date) as month_num
            ,sum(amount) as total_revenue
        from orders
        group by month(date)
        order by month(date)) as t

""").show()

+---------+-------------+----+------------------+
|month_num|total_revenue| lag|        growth_per|
+---------+-------------+----+------------------+
|        5|         2425|NULL|              NULL|
|        6|         3220|2425| 32.78350515463917|
|        7|         4845|3220|50.465838509316775|
+---------+-------------+----+------------------+

